In [ ]:
# Importing all the Librarys requried

In [6]:
import math
import random as rd
import numpy as np
import pandas as pd
import regex as re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy as s
import scipy.stats as ss
from sklearn.utils import resample
import warnings 
warnings.filterwarnings('ignore')
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,LancasterStemmer,SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import emoji
from textblob import TextBlob

In [ ]:
# converting the CSV to DataFrame

In [2]:
data=pd.read_csv(r"C:\Users\Gogula Vinay\Desktop\Innomatics labs\Data Science\Machine Learning\ML Assignment 2\nyka_top_brands_cosmetics_product_reviews.csv")
data.head()

,product_id,brand_name,review_id,review_title,review_text,author,review_date,review_rating,is_a_buyer,pro_user,review_label,product_title,mrp,price,product_rating,product_rating_count,product_tags,product_url
0,781070,Olay,16752142,Worth buying 50g one,Works as it claims. Could see the difference f...,Ashton Dsouza,2021-01-23 15:17:18,5.0,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...
1,781070,Olay,14682550,Best cream to start ur day,It does what it claims . Best thing is it smoo...,Amrit Neelam,2020-09-07 15:30:42,5.0,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...
2,781070,Olay,15618995,perfect for summers dry for winters,I have been using this product for months now....,Sanchi Gupta,2020-11-13 12:24:14,4.0,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...
3,781070,Olay,13474509,Not a moisturizer,"i have an oily skin, while this whip acts as a...",Ruchi Shah,2020-06-14 11:56:50,3.0,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...
4,781070,Olay,16338982,Average,It's not that good. Please refresh try for oth...,Sukanya Sarkar,2020-12-22 15:24:35,2.0,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...


In [3]:
data.columns

Index(['product_id', 'brand_name', 'review_id', 'review_title', 'review_text',
       'author', 'review_date', 'review_rating', 'is_a_buyer', 'pro_user',
       'review_label', 'product_title', 'mrp', 'price', 'product_rating',
       'product_rating_count', 'product_tags', 'product_url'],
      dtype='object')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61284 entries, 0 to 61283
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   product_id            61284 non-null  int64  
 1   brand_name            61284 non-null  object 
 2   review_id             61284 non-null  int64  
 3   review_title          61284 non-null  object 
 4   review_text           61276 non-null  object 
 5   author                61284 non-null  object 
 6   review_date           61284 non-null  object 
 7   review_rating         61283 non-null  float64
 8   is_a_buyer            61284 non-null  bool   
 9   pro_user              61284 non-null  bool   
 10  review_label          48249 non-null  object 
 11  product_title         61284 non-null  object 
 12  mrp                   61284 non-null  int64  
 13  price                 61284 non-null  int64  
 14  product_rating        61284 non-null  float64
 15  product_rating_coun

In [5]:
data.describe()

,product_id,review_id,review_rating,mrp,price,product_rating,product_rating_count
count,6.128400e+04,6.128400e+04,61283.000000,61284.000000,61284.000000,61284.000000,61284.000000
mean,7.983802e+05,1.484995e+07,4.414781,573.260247,462.129512,4.099130,7582.963840
std,1.281418e+06,7.383506e+06,1.062547,324.098930,264.876964,0.235945,14463.246136
min,2.500000e+02,9.600000e+01,1.000000,75.000000,45.000000,1.500000,1.000000
25%,1.604880e+05,1.102373e+07,4.000000,300.000000,262.000000,4.000000,1760.000000
50%,4.524430e+05,1.525142e+07,5.000000,599.000000,400.000000,4.100000,3925.000000
75%,7.665290e+05,2.002927e+07,5.000000,799.000000,639.000000,4.300000,8720.000000
max,7.749427e+06,2.963031e+07,5.000000,3874.000000,2947.000000,4.800000,98477.000000


In [6]:
data.T

,0,1,2,3,4,5,6,7,8,9,...,61274,61275,61276,61277,61278,61279,61280,61281,61282,61283
product_id,781070,781070,781070,781070,781070,781070,739418,739418,739418,739418,...,1044490,1044490,1044490,1044490,1044490,1044490,1044490,1044490,1044490,4633369
brand_name,Olay,Olay,Olay,Olay,Olay,Olay,Olay,Olay,Olay,Olay,...,Herbal Essences,Herbal Essences,Herbal Essences,Herbal Essences,Herbal Essences,Herbal Essences,Herbal Essences,Herbal Essences,Herbal Essences,Colorbar
review_id,16752142,14682550,15618995,13474509,16338982,14549640,16531371,21356560,15235570,22008691,...,17833518,17810994,17801684,17800493,17798807,17769058,17766838,17765866,17761057,25987603
review_title,Worth buying 50g one,Best cream to start ur day,perfect for summers dry for winters,Not a moisturizer,Average,not good for oily skin,All time favorite,"""Good Product """,Good eye cream combo,"""Olay''",...,awesome shampoo,Perfect shampoo,Awesome product,It is a good one!,Just amazing,Good,Great,Really mild and nice,Amazing product!,Not good
review_text,Works as it claims. Could see the difference f...,It does what it claims . Best thing is it smoo...,I have been using this product for months now....,"i have an oily skin, while this whip acts as a...",It's not that good. Please refresh try for oth...,dz product z best for dry skin ...one of olay ...,"This cream is just awesome, It makes my rough ...",Instantly perfect skin tone appearance.,This eye cream combo is effective. Works on fi...,"3in1 benifits, helps reduces dark spots and wr...",...,it truly makes my hair soft and bouncy plus it...,This shampoo is free from chemicals and parabe...,I was experiencing extremely dry hair so thoug...,"It is a huge bottle, will easily last for a ve...",This shampoo makes my hair soft and retains mo...,I use this thrice a week. Has helped me in mak...,This shampoo has a amazing fragnance and textu...,"Used it for the first time, feels really fresh...",One of the best shampoo which are sulphate and...,It is so irritating so bad i don't like this much
author,Ashton Dsouza,Amrit Neelam,Sanchi Gupta,Ruchi Shah,Sukanya Sarkar,Laxmi Basumatary,Priyanka Barwal,Bandana Mukherjee,krish,Gargi Mukherjee,...,Pooja G,Diksha Chauhan,Sachin,lashita dhingra,Meenal Mehta,Radhesh,Shalini Sirohi,Sana kanted,neeru chouhan,Mssuhv
review_date,2021-01-23 15:17:18,2020-09-07 15:30:42,2020-11-13 12:24:14,2020-06-14 11:56:50,2020-12-22 15:24:35,2020-08-27 18:16:47,2021-01-06 15:43:25,2021-11-13 19:57:06,2020-10-19 15:08:13,2021-12-16 14:55:52,...,2021-04-16 15:51:59,2021-04-15 01:46:58,2021-04-14 13:32:20,2021-04-14 12:36:13,2021-04-14 11:06:19,2021-04-12 13:28:41,2021-04-12 11:47:07,2021-04-12 10:58:34,2021-04-11 21:44:55,2022-06-20 17:41:52
review_rating,5.0,5.0,4.0,3.0,2.0,1.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,2.0
is_a_buyer,True,True,True,True,True,True,False,False,False,False,...,True,True,True,True,True,True,True,True,True,False
pro_user,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
data.isnull().sum()

product_id                  0
brand_name                  0
review_id                   0
review_title                0
review_text                 8
author                      0
review_date                 0
review_rating               1
is_a_buyer                  0
pro_user                    0
review_label            13035
product_title               0
mrp                         0
price                       0
product_rating              0
product_rating_count        0
product_tags            47782
product_url                 0
dtype: int64

In [8]:
# indipendent columns

In [26]:
data[['review_text']]

,review_text
0,Works as it claims. Could see the difference f...
1,It does what it claims . Best thing is it smoo...
2,I have been using this product for months now....
3,"i have an oily skin, while this whip acts as a..."
4,It's not that good. Please refresh try for oth...
...,...
61279,I use this thrice a week. Has helped me in mak...
61280,This shampoo has a amazing fragnance and textu...
61281,"Used it for the first time, feels really fresh..."
61282,One of the best shampoo which are sulphate and...


In [27]:
data[['review_text']].isnull().sum()

review_text    0
dtype: int64

In [28]:
data['review_text'].unique()

array(['Works as it claims. Could see the difference from the first day. Use it with Olay cleanser for best results',
       'It does what it claims . Best thing is it smoothens ur skin n makes it soft . I liked it',
       'I have been using this product for months now.. it is perfect for combination n oily skin as it is non greasy absorbs quickly and moisturises well but it doesnt work for winters',
       ...,
       'Used it for the first time, feels really fresh after using. I have an itchy scalp n face dandruff problem. In one wash dandruff got reduced. It cleanses well and smells nice.',
       'One of the best shampoo which are sulphate and Paraben free. It makes my hair looks more shiny and moistured. I am satisfied my the quality of the product.',
       "It is so irritating so bad i don't like this much"], dtype=object)

In [29]:
data.fillna({'review_text':'revtext'},inplace=True)


In [30]:
data[(data['review_text']=='revtext')]

,product_id,brand_name,review_id,review_title,review_text,author,review_date,review_rating,is_a_buyer,pro_user,review_label,product_title,mrp,price,product_rating,product_rating_count,product_tags,product_url


In [31]:
a=data[(data['review_text']=='revtext')].index
a

Int64Index([], dtype='int64')

In [32]:
data.drop(a,inplace=True)

In [33]:
data[(data['review_text']=='revtext')]

,product_id,brand_name,review_id,review_title,review_text,author,review_date,review_rating,is_a_buyer,pro_user,review_label,product_title,mrp,price,product_rating,product_rating_count,product_tags,product_url


In [34]:
data['review_text'].isnull().sum()

0

In [22]:
data

,product_id,brand_name,review_id,review_title,review_text,author,review_date,review_rating,is_a_buyer,pro_user,review_label,product_title,mrp,price,product_rating,product_rating_count,product_tags,product_url
0,781070,Olay,16752142,Worth buying 50g one,Works as it claims. Could see the difference f...,Ashton Dsouza,2021-01-23 15:17:18,Positive,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...
1,781070,Olay,14682550,Best cream to start ur day,It does what it claims . Best thing is it smoo...,Amrit Neelam,2020-09-07 15:30:42,Positive,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...
2,781070,Olay,15618995,perfect for summers dry for winters,I have been using this product for months now....,Sanchi Gupta,2020-11-13 12:24:14,Positive,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...
4,781070,Olay,16338982,Average,It's not that good. Please refresh try for oth...,Sukanya Sarkar,2020-12-22 15:24:35,Negative,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...
5,781070,Olay,14549640,not good for oily skin,dz product z best for dry skin ...one of olay ...,Laxmi Basumatary,2020-08-27 18:16:47,Negative,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61279,1044490,Herbal Essences,17769058,Good,I use this thrice a week. Has helped me in mak...,Radhesh,2021-04-12 13:28:41,Positive,True,False,Verified Buyer,Herbal Essences Aloe & Eucalyptus Shampoo For ...,750,525,4.3,512,FEATURED,https://www.nykaa.com/herbal-essences-potent-a...
61280,1044490,Herbal Essences,17766838,Great,This shampoo has a amazing fragnance and textu...,Shalini Sirohi,2021-04-12 11:47:07,Positive,True,False,Verified Buyer,Herbal Essences Aloe & Eucalyptus Shampoo For ...,750,525,4.3,512,FEATURED,https://www.nykaa.com/herbal-essences-potent-a...
61281,1044490,Herbal Essences,17765866,Really mild and nice,"Used it for the first time, feels really fresh...",Sana kanted,2021-04-12 10:58:34,Positive,True,False,Verified Buyer,Herbal Essences Aloe & Eucalyptus Shampoo For ...,750,525,4.3,512,FEATURED,https://www.nykaa.com/herbal-essences-potent-a...
61282,1044490,Herbal Essences,17761057,Amazing product!,One of the best shampoo which are sulphate and...,neeru chouhan,2021-04-11 21:44:55,Positive,True,False,Verified Buyer,Herbal Essences Aloe & Eucalyptus Shampoo For ...,750,525,4.3,512,FEATURED,https://www.nykaa.com/herbal-essences-potent-a...


In [26]:
stp=stopwords.words("english")

In [27]:
stp.remove("not")

In [28]:
stemming=input('enter the stemming you need to perform: ')
lf1=[]
for sent in data['review_text']:
        l=[]
        ls=sent.lower()
        ls=emoji.demojize(ls)
        ls=re.sub('<.*?>','',ls)
        ls=re.sub('https?://\S*|www\S*','',ls)
        ls=re.sub("[^a-z0-9 ]","",ls)
        for word in word_tokenize(ls):
            if word in stp:
                pass
            else:
                if stemming=='PorterStemmer':
                            stems=PorterStemmer().stem(word)
                            l.append(stems)
                elif stemming=='LancasterStemmer':
                            stems=LancasterStemmer().stem(word)
                            l.append(stems)
                elif stemming=='SnowballStemmer':
                            stems=SnowballStemmer("english").stem(word)
                            l.append(stems) 
                elif stemming=='WordNetLemmatizer':
                            stems=WordNetLemmatizer().lemmatize(word)
                            l.append(stems)
        lf1.append(" ".join(l))

enter the stemming you need to perform: PorterStemmer


In [29]:
cleaned_data=pd.DataFrame({"cleand_review":lf1})

In [30]:
cleaned_data

,cleand_review
0,work claim could see differ first day use olay...
1,claim best thing smoothen ur skin n make soft ...
2,use product month perfect combin n oili skin n...
3,oili skin whip act great base primer smoothen ...
4,not good pleas refresh tri product
...,...
61271,use thrice week help make hair smooth hair rou...
61272,shampoo amaz fragnanc textur like aloevera cle...
61273,use first time feel realli fresh use itchi sca...
61274,one best shampoo sulphat paraben free make hai...


In [31]:
cv=CountVectorizer()
dt=cv.fit_transform(cleaned_data["cleand_review"])
dt

<61276x21391 sparse matrix of type '<class 'numpy.int64'>'
	with 678599 stored elements in Compressed Sparse Row format>

In [32]:
pd.DataFrame(dt.toarray(),columns=sorted(cv.vocabulary_.keys()))

,00001,001,01,0100,01suitabl,02,028,02goe,02nice,03,...,zipper,zippermouthfac,zist,zit,zizt,zoc,zombi,zone,zoom,zoster
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61271,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61272,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61273,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
#tfidf

In [34]:
tf=TfidfVectorizer()

In [35]:
cond=tf.fit_transform(cleaned_data["cleand_review"])
cond

<61276x21391 sparse matrix of type '<class 'numpy.float64'>'
	with 678599 stored elements in Compressed Sparse Row format>

In [36]:
pd.DataFrame(cond.toarray(),columns=sorted(tf.vocabulary_.keys()))

,00001,001,01,0100,01suitabl,02,028,02goe,02nice,03,...,zipper,zippermouthfac,zist,zit,zizt,zoc,zombi,zone,zoom,zoster
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61271,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
61274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
# One-Hot Encoding

In [7]:
data=pd.read_csv(r"C:\Users\Gogula Vinay\Desktop\Innomatics labs\Data Science\Machine Learning\ML Assignment 2\nyka_top_brands_cosmetics_product_reviews.csv")
data.head()

,product_id,brand_name,review_id,review_title,review_text,author,review_date,review_rating,is_a_buyer,pro_user,review_label,product_title,mrp,price,product_rating,product_rating_count,product_tags,product_url
0,781070,Olay,16752142,Worth buying 50g one,Works as it claims. Could see the difference f...,Ashton Dsouza,2021-01-23 15:17:18,5.0,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...
1,781070,Olay,14682550,Best cream to start ur day,It does what it claims . Best thing is it smoo...,Amrit Neelam,2020-09-07 15:30:42,5.0,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...
2,781070,Olay,15618995,perfect for summers dry for winters,I have been using this product for months now....,Sanchi Gupta,2020-11-13 12:24:14,4.0,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...
3,781070,Olay,13474509,Not a moisturizer,"i have an oily skin, while this whip acts as a...",Ruchi Shah,2020-06-14 11:56:50,3.0,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...
4,781070,Olay,16338982,Average,It's not that good. Please refresh try for oth...,Sukanya Sarkar,2020-12-22 15:24:35,2.0,True,False,Verified Buyer,Olay Ultra Lightweight Moisturiser: Luminous W...,1999,1599,4.1,43,NaN,https://www.nykaa.com/olay-ultra-lightweight-m...


In [8]:
data['review_rating'].unique()

array([ 5.,  4.,  3.,  2.,  1., nan])

In [9]:
data[['review_rating']].isnull().sum()

review_rating    1
dtype: int64

In [10]:
data.fillna({'review_rating':'revrating'},inplace=True)
a=data[(data['review_rating']=='revrating')].index
a

Int64Index([21576], dtype='int64')

In [11]:
data.drop(a,inplace=True)

In [12]:
data['review_rating']=data['review_rating'].apply(int)
data['review_rating'].unique()

array([5, 4, 3, 2, 1], dtype=int64)

In [13]:
indexgreat3=data[data['review_rating']>3].index

In [14]:
indexequal3=data[data['review_rating']==3].index

In [15]:
indexless3=data[data['review_rating']<3].index

In [16]:
for i in indexless3:
    data['review_rating'][i]='Negative'

In [17]:
for i in indexgreat3:
    data['review_rating'][i]='Positive'

In [18]:
data.drop(indexequal3,inplace=True)

In [19]:
data['review_rating'].unique()

array(['Positive', 'Negative'], dtype=object)

In [20]:
# One-Hot Encoding

In [21]:
oh=OneHotEncoder()
coverted=oh.fit_transform(data[['review_rating']])
pd.DataFrame(coverted.toarray(),columns=oh.categories_).applymap(int)

,Negative,Positive
0,0,1
1,0,1
2,0,1
3,1,0
4,1,0
...,...,...
57738,0,1
57739,0,1
57740,0,1
57741,0,1
